<a href="https://colab.research.google.com/github/sharon220596/python/blob/main/langchain_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install langchain-openai langchain-core langchain-community python-dotenv
!pip install faiss-cpu
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.4/476.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.1
    Uninstalling langchain-core-1.2.1:
      Successfully uninstalled langchain-core-1.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-co

## Make a simple call

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the Prompt Template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{text}")
])

# LangChain automatically uses the API key from the environment variable
openai = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
gemini = None
# Initialize the Output Parser
output_parser = StrOutputParser()

# Build the LCEL Chain using the pipe (|) operator
chain = prompt | openai | output_parser

# Invoke the chain
result = chain.invoke({
    "input_language": "English",
    "output_language": "Hindi",
    "text": "I love programming with LangChain!"
})


print(result)




RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from google.colab import userdata # Assuming you are still in Colab

# --- Setup (Install libraries if needed, set API key) ---
# !pip install -qqq langchain-openai langchain-core langchain-community

# 1. Model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

# 2. Prompt (Use MessagesPlaceholder for history injection)
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant."),
    # This placeholder handles the history correctly in v1.x
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# 3. Core Chain & Memory Store
store = {} # A simple in-memory dictionary acting as our database
output_parser = StrOutputParser()

# The core stateless chain built with LCEL
core_chain = prompt | llm | output_parser

# Function to provide the correct history object based on session ID
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 4. Wrap the chain with the v1.x history manager
chain_with_history = RunnableWithMessageHistory(
    core_chain,
    get_session_history,
    input_messages_key="input", # Key for the new human input
    history_messages_key="chat_history", # Key used in the prompt template placeholder
)
# 5. Interactions
session_id = "test_session_456"
config = {"configurable": {"session_id": session_id}}

# Interaction 1: Pass a dictionary with the "input" key
r1 = chain_with_history.invoke(
    {"input": "Hi, I'm Roopesh"}, # Corrected input format
    config=config
)
print(f"AI 1st: {r1}")

# Interaction 2: Pass a dictionary again
r2 = chain_with_history.invoke(
    {"input": "What's my name?"}, # Corrected input format
    config=config
)
print(f"AI 2nd: {r2}")

# The store dictionary now holds the conversation history correctly
print("\nFull conversation history:", store[session_id].messages)

#Full conversation history: [HumanMessage(content="Hi, I'm Roopesh", additional_kwargs={}, response_metadata={}), AIMessage(content='Hi Roopesh! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Roopesh! How can I help you today?', additional_kwargs={}, response_metadata={})]

#[26]
#2s


AI 1st: Hello, Roopesh! How can I assist you today?
AI 2nd: Your name is Roopesh! How can I help you today?

Full conversation history: [HumanMessage(content="Hi, I'm Roopesh", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Roopesh! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Roopesh! How can I help you today?', additional_kwargs={}, response_metadata={})]


In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize components
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
parser = StrOutputParser()

# Single chained statement
result = (
    ChatPromptTemplate.from_template("Translate to German: {text}") | llm | parser |
    (lambda x: {"text": x}) |  # Wrap output as dict for next chain
    ChatPromptTemplate.from_template("Make this exciting, but in simple language: {text}") | llm | parser |
    (lambda x: {"text": x}) |  # Wrap again
    ChatPromptTemplate.from_template("Say this like actor Amitabh Bachchan: {text}") | llm | parser
).invoke({"text": "The future of AI is exciting"})

print(result)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
result = chain.invoke({"input_language": "English",
                       "output_language": "Spanish",
                       "text": "Hello"})
print(result)

# 2. batch() - Multiple calls in parallel
results = chain.batch([
    {"input_language": "English", "output_language": "French", "text": "Hello"},
    {"input_language": "English", "output_language": "German", "text": "Hello"},
    {"input_language": "English", "output_language": "Italian", "text": "Hello"}
])
for r in results:
    print(r)

# 3. stream() - Token by token streaming
for chunk in chain.stream({"input_language": "English",
                           "output_language": "Japanese",
                           "text": "Hello"}):
    print(chunk, end="", flush=True)

Hola
Bonjour
Hallo
Ciao
こんにちは (Konnichiwa)

In [ ]:
chain_with_history = RunnableWithMessageHistory(
    core_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 6. Have conversations!
session_id = "user_123"
config = {"configurable": {"session_id": session_id}}

# First interaction
response1 = chain_with_history.invoke(
    {"input": "Hi, I'm Roopesh"},
    config=config
)
print(f"AI: {response1}")
# Output: "Hello Roopesh! Nice to meet you. How can I help you today?"

# Second interaction - AI remembers!
response2 = chain_with_history.invoke(
    {"input": "What's my name?"},
    config=config
)
print(f"AI: {response2}")
# Output: "Your name is Roopesh!"

# Check stored history
print("\nHistory:", store[session_id].messages)

AI: Hello, Roopesh! How can I assist you today?
AI: Your name is Roopesh. How can I help you today?

History: [HumanMessage(content="Hi, I'm Roopesh", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Roopesh! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Roopesh. How can I help you today?', additional_kwargs={}, response_metadata={})]


In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader

# 1. Load and split documents
loader = TextLoader("company_policies.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)

# 2. Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
# 3. Define prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer based on the following context:\n\n{context}"),
    ("human", "{question}")
])

# 4. Helper function to format docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 5. Build the RAG chain with LCEL
model = ChatOpenAI(model="gpt-4o-mini")

rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | prompt
    | model
    | StrOutputParser()
)

# 6. Ask questions!
answer = rag_chain.invoke("What is the remote work policy?")
print(answer)

The remote work policy allows employees to work remotely for two days per week, choosing any two days they prefer. Additionally, women are permitted to work remotely for three days per week.


In [ ]:
#from langgraph.prebuilt import create_react_agent
from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a city."""
    return f"It's always sunny in {city}!"

# Create agent with tools
agent = create_agent(
    model="gpt-3.5-turbo",
    tools=[get_weather],
)

# Run the agent
result = agent.invoke({
    "messages": [{"role": "user", "content": "what is the weather in sf"}]
})



[HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='90cbadc5-c799-4ea4-9d22-c2fcaca83277'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 50, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Cj3CdoNpUNIKw3pCzYaRJO1zqRPLT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f4ff3f0e-34cd-4447-b7d4-2312899339c7-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_IAr1G6IqIV4TKtGUbeCigBcz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 50, 'output_tokens': 15, 'total_tokens': 65, 'input_token_d

## RAG

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# 1. Load and split documents
loader = TextLoader("company_policies.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)

# 2. Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})


# 3. Define prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer based on the following context:\n\n{context}"),
    ("human", "{question}")
])

# 4. Helper function to format docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 5. Build the RAG chain with LCEL
model = ChatOpenAI(model="gpt-4o-mini")

rag_chain = (
    RunnableParallel({
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    })
    | prompt
    | model
    | StrOutputParser()
)

# 6. Ask questions!
#answer = rag_chain.invoke("What is the remote work policy?")
#print(answer)

# Stream the response token by token, Output streams live as it's generated!
for chunk in rag_chain.stream("What is the remote work policy?"):
    print(chunk, end="", flush=True)
#How many leaves can be availed by an employee?


The remote work policy at Ojasa Mirai LLP allows employees to work remotely up to 2 days per week with manager approval. Remote work days must be scheduled in advance through the HR portal. Additionally, women are permitted a total of 10 days of work-from-home, not exceeding 3 days in a week.

In [ ]:
answer

'The remote work policy allows employees to work remotely up to 2 days per week with manager approval. Remote work days must be scheduled in advance through the HR portal. Additionally, women are permitted a total of 10 days of work-from-home, but cannot exceed 3 days in a single week.'

## Lang Graph - Agent

In [ ]:
# LangGraph ReAct Agent with OpenAI
# This example shows how to create a simple agent that can use tools

from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# Define a tool using the @tool decorator
@tool
def get_weather(city: str) -> str:
    """Get weather for a city. Use this when user asks about weather."""
    return f"It's always sunny in {city}!"

# Create the OpenAI model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Create agent with tools
# Note: The prompt parameter is used for system message
agent = create_react_agent(
    model=model,
    tools=[get_weather]
)

# Run the agent
print("=" * 60)
print("LANGGRAPH REACT AGENT DEMO WITH OPENAI")
print("=" * 60)

result = agent.invoke({
    "messages": [{"role": "user", "content": "what is the weather in sf"}]
})

# Print the conversation
print("\nConversation:")
print("-" * 60)
for message in result["messages"]:
    if hasattr(message, 'content'):
        role = message.__class__.__name__
        print(f"\n{role}:")
        print(message.content)
    if hasattr(message, 'tool_calls') and message.tool_calls:
        print(f"\nTool Calls: {message.tool_calls}")
print("\n" + "=" * 60)



/tmp/ipython-input-2300434243.py:19: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


LANGGRAPH REACT AGENT DEMO WITH OPENAI

Conversation:
------------------------------------------------------------

HumanMessage:
what is the weather in sf

AIMessage:


Tool Calls: [{'name': 'get_weather', 'args': {'city': 'San Francisco'}, 'id': 'call_wEBZBP4fyPmPUJDi7U1qGHUc', 'type': 'tool_call'}]

ToolMessage:
It's always sunny in San Francisco!

AIMessage:
The weather in San Francisco is always sunny!



In [ ]:
# Try more examples
print("\n\nMORE EXAMPLES:")
print("=" * 60)

examples = [
    "What's the weather in New York?",
    "Compare weather in Paris and London",
    "Tell me about the weather in Tokyo"
]

for question in examples:
    print(f"\n\nUser: {question}")
    result = agent.invoke({
        "messages": [{"role": "user", "content": question}]
    })
    # Get the final response
    final_message = result["messages"][-1]
    print(f"Agent: {final_message.content}")

## Other ex

In [ ]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt Template - be very specific to avoid web search explanations
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a translator. Translate the text from {input_language} to {output_language}. Return ONLY the translated text, no explanations or additional information."),
    ("user", "{text}")
])

# Langchain automatically uses the API key from the environment variable

# initialize the output parser
output_parser = StrOutputParser()

# Build the LCEL chain using the (|) pipe operator
chain = prompt | openai | output_parser

# Invoke the chain
result = chain.invoke({
    "input_language" : "English",
    "output_language": "Spanish",
    "text" : "Hello , How is it going?"
})

print(result)

Hola, ¿cómo estás?


Multi-agent

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.4)
output_parser = StrOutputParser()

research_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a research agent. Study the given topic and produce crisp bullet insights."),
    ("human", "Topic: {topic}")
])
research_chain = research_prompt | model | output_parser

writer_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a writing agent. Draft a clean and well-structured article based only on the research findings. Avoid repetition and fluff."),
    ("human", "Research findings:\n{research_data}")
])
writer_chain = writer_prompt | model | output_parser

def orchestrate(topic: str):
    research_summary = research_chain.invoke({"topic": topic})
    article = writer_chain.invoke({"research_data": research_summary})
    return {"research": research_summary, "article": article}

if __name__ == "__main__":
    topic = "Future of AI in retail inventory and automated logistics"
    result = orchestrate(topic)
    print("\n================ RESEARCH OUTPUT ================\n")
    print(result["research"])
    print("\n================ FINAL ARTICLE ================\n")
    print(result["article"])


In [ ]:

from langchain.agents import create_agent, create_
from langchain_core.tools import tool
model = ChatOpenAI(model="gpt-4.1-mini", temperature=0.4)

@tool
def pull_from_history(topic:str) -> str:
  pass

@tool
def research(topic: str) -> str:
    """This is for research agent"""
    # Create the prompt with proper formatting
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a research agent, please get the facts in detailed text under 500 words."),
        ("human", "research topic: {input_topic}")
    ])

    # Format the prompt with the topic
    prompt = prompt_template.format_messages(input_topic=topic)

    # Call the model
    output = model.invoke(prompt)
    return output.content if hasattr(output, 'content') else str(output)

@tool
def writing(research_data: str) -> str:
    """This is for writing agent"""
    # Create the prompt template
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "You are a tech writer. Write the technical article in a national newspaper in India under 100 words."),
        ("human", "Please stick to the Research findings:\n{input_data}")
    ])

    # Format the prompt with the research data
    prompt = prompt_template.format_messages(input_data=research_data)

    # Call the model
    output = model.invoke(prompt)
    return output.content if hasattr(output, 'content') else str(output)

agent_research=create_agent(
    model=model,
    tools=[research, pull_from_history]
)
agent_writer=create_agent(
    model=model,
    tools=[writing]
)

topic = {"human": "increased influence of AI in education"}
output = agent_research.invoke(topic)
print (output)
## Orchestration


TypeError: create_agent() got an unexpected keyword argument 'llm'

# Multi-agent orchestrator

Multi-Agent System Practice Code - LangChain/LangGraph
=======================================================

This code demonstrates a complete multi-agent system for content creation,
following the supervisor/orchestrator pattern with specialized agents.

Business Context: Content Creation Pipeline
- Research Agent: Gathers information and sources
- Outline Agent: Creates structured content outline
- Writing Agent: Drafts the actual content
- Editor Agent: Reviews and improves quality
- Coordinator Agent: Orchestrates the entire workflow

Based on the slide content: Multi-Agent Design

## Setup

In [ ]:

import os
from typing import TypedDict, Annotated, List, Dict, Any
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import tool, ToolRuntime
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from datetime import datetime


# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

# Initialize LLM with appropriate model
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)



## Tools

In [ ]:
@tool
def web_search_tool(query: str) -> str:
    """
    Simulates web search to gather information.
    """
    # Simulated search results
    search_results = {
        "AI": "AI (Artificial Intelligence) refers to systems that can perform tasks requiring human intelligence. Key developments include machine learning, neural networks, and large language models.",
        "climate": "Climate change refers to long-term shifts in temperatures and weather patterns. Main causes include greenhouse gas emissions from fossil fuels.",
        "technology": "Technology encompasses tools, systems, and methods used to solve problems. Recent trends include AI, cloud computing, and IoT.",
        "default": f"Information about {query}: This is a simulated search result. In production, this would return real web data."
    }

    # Simple keyword matching for simulation
    for keyword, result in search_results.items():
        if keyword.lower() in query.lower():
            return result

    return search_results["default"]


@tool
def fact_checker_tool(statement: str) -> Dict[str, Any]:
    """
    Checks facts and provides verification status.
    In production, this would use fact-checking APIs.

    Args:
        statement: The statement to verify

    Returns:
        Dictionary with verification status and details
    """
    # Simulated fact checking
    return {
        "statement": statement,
        "verified": True,
        "confidence": 0.85,
        "sources": ["Simulated Source 1", "Simulated Source 2"],
        "notes": "This is a simulated fact check. In production, use real verification services."
    }


@tool
def grammar_checker_tool(text: str) -> Dict[str, Any]:
    """
    Checks grammar and style in the provided text.
    In production, this would use services like Grammarly API.

    Args:
        text: The text to check

    Returns:
        Dictionary with grammar check results
    """
    # Simulated grammar checking
    word_count = len(text.split())

    return {
        "text_length": len(text),
        "word_count": word_count,
        "issues_found": 0,  # Simulated
        "readability_score": 8.5,  # Simulated (out of 10)
        "suggestions": ["Great job! Text appears well-written."],
        "status": "approved"
    }


@tool
def outline_generator_tool(topic: str, research_data: str) -> str:
    """
    Generates a structured outline based on research data.

    Args:
        topic: The main topic
        research_data: Research information gathered

    Returns:
        Structured outline as a string
    """
    outline = f"""
CONTENT OUTLINE: {topic}

I. Introduction
   - Hook: Engaging opening about {topic}
   - Context: Why this matters
   - Thesis: Main argument

II. Background & Research
   {research_data[:200]}...

III. Main Points
   - Point 1: Key concept
   - Point 2: Supporting evidence
   - Point 3: Practical implications

IV. Conclusion
   - Summary of key points
   - Call to action
   - Future outlook
"""
    return outline




## Agents

### Research Agent

In [ ]:

class ResearchAgent:
    """
    Agent 1: Research Agent
    Responsibility: Searches for information and gathers sources
    """

    def __init__(self, llm):
        self.llm = llm
        self.tools = [web_search_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a Research Agent specialized in gathering information.

Your tasks:
1. Use the web_search_tool to find relevant information
2. Identify key points and facts
3. Organize findings in a clear, structured way
4. Cite sources when available

Always be thorough and accurate in your research."""),
            ("human", "{task}")
        ])

    def execute(self, task: str) -> str:
        """Execute research task"""
        # First, use the search tool
        search_query = f"research information about: {task}"
        search_results = web_search_tool.invoke({"query": search_query})

        # Then, have the LLM process and organize the results
        chain = self.prompt | self.llm | StrOutputParser()

        enhanced_task = f"""
Task: {task}

Search Results:
{search_results}

Please organize these findings into a clear research summary with key points.
"""

        result = chain.invoke({"task": enhanced_task})
        return result




### OutlineAgent

In [ ]:
class OutlineAgent:
    """
    Agent 2: Outline Agent
    Responsibility: Creates structured content outline from research
    """

    def __init__(self, llm):
        self.llm = llm
        self.tools = [outline_generator_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an Outline Agent specialized in creating structured content.

Your tasks:
1. Analyze research data
2. Create logical content structure
3. Organize main points hierarchically
4. Ensure flow and coherence

Create clear, actionable outlines that guide content creation."""),
            ("human", "{research_data}\n\nTopic: {topic}")
        ])

    def execute(self, topic: str, research_data: str) -> str:
        """Execute outline creation"""
        # Use the outline generator tool
        outline = outline_generator_tool.invoke({
            "topic": topic,
            "research_data": research_data
        })

        # Have the LLM enhance it
        chain = self.prompt | self.llm | StrOutputParser()

        result = chain.invoke({
            "topic": topic,
            "research_data": f"Research:\n{research_data}\n\nInitial Outline:\n{outline}"
        })

        return result



### WritingAgent

In [ ]:
class WritingAgent:
    """
    Agent 3: Writing Agent
    Responsibility: Drafts content based on outline and research
    """

    def __init__(self, llm):
        self.llm = llm
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a Writing Agent specialized in creating engaging content.

Your tasks:
1. Follow the provided outline structure
2. Incorporate research findings naturally
3. Write in a clear, engaging style
4. Maintain consistent tone and voice
5. Create smooth transitions between sections

Write content that is informative, engaging, and well-structured."""),
            ("human", """Create content based on:

OUTLINE:
{outline}

RESEARCH DATA:
{research_data}

Topic: {topic}""")
        ])

    def execute(self, topic: str, outline: str, research_data: str) -> str:
        """Execute content writing"""
        chain = self.prompt | self.llm | StrOutputParser()

        result = chain.invoke({
            "topic": topic,
            "outline": outline,
            "research_data": research_data
        })

        return result


class EditorAgent:
    """
    Agent 4: Editor Agent
    Responsibility: Reviews and improves content quality
    """

    def __init__(self, llm):
        self.llm = llm
        self.tools = [grammar_checker_tool, fact_checker_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an Editor Agent specialized in quality assurance.

Your tasks:
1. Check grammar and style
2. Verify facts and accuracy
3. Improve clarity and readability
4. Ensure consistency
5. Provide constructive feedback

Be thorough but constructive in your review."""),
            ("human", """Review this content:

{content}

Provide:
1. Quality assessment
2. Suggestions for improvement
3. Final recommendation (approve/revise)""")
        ])

    def execute(self, content: str) -> Dict[str, Any]:
        """Execute content review"""
        # Use grammar checker
        grammar_check = grammar_checker_tool.invoke({"text": content})

        # Use fact checker on key claims (simplified for demo)
        # In production, you'd extract claims and check each

        # Have LLM provide editorial review
        chain = self.prompt | self.llm | StrOutputParser()

        editorial_review = chain.invoke({"content": content})

        return {
            "grammar_check": grammar_check,
            "editorial_review": editorial_review,
            "status": grammar_check["status"],
            "timestamp": datetime.now().isoformat()
        }


### COORDINATOR/ORCHESTRATOR AGENT

In [ ]:
class CoordinatorAgent:
    """
    Main Coordinator Agent (Supervisor Pattern)
    Responsibility: Orchestrates the entire multi-agent workflow

    This implements the "Tool Calling" pattern from LangChain where
    the coordinator treats other agents as tools to invoke.
    """

    def __init__(self, llm):
        self.llm = llm

        # Initialize all specialized agents
        self.research_agent = ResearchAgent(llm)
        self.outline_agent = OutlineAgent(llm)
        self.writing_agent = WritingAgent(llm)
        self.editor_agent = EditorAgent(llm)

        # Track workflow state
        self.workflow_state = {}

    def orchestrate_content_creation(self, topic: str) -> Dict[str, Any]:

        print(f"\n{'='*70}")
        print(f"🤖 MULTI-AGENT CONTENT CREATION PIPELINE")
        print(f"{'='*70}")
        print(f"Topic: {topic}\n")

        workflow_results = {
            "topic": topic,
            "start_time": datetime.now().isoformat(),
            "stages": {}
        }

        # STAGE 1: Research
        print("📚 Stage 1: Research Agent - Gathering Information...")
        research_data = self.research_agent.execute(topic)
        workflow_results["stages"]["research"] = {
            "agent": "ResearchAgent",
            "output": research_data,
            "status": "completed"
        }
        print(f"✓ Research completed ({len(research_data)} characters)\n")

        # STAGE 2: Outline
        print("📝 Stage 2: Outline Agent - Creating Structure...")
        outline = self.outline_agent.execute(topic, research_data)
        workflow_results["stages"]["outline"] = {
            "agent": "OutlineAgent",
            "output": outline,
            "status": "completed"
        }
        print(f"✓ Outline created\n")

        # STAGE 3: Writing
        print("✍️  Stage 3: Writing Agent - Drafting Content...")
        content = self.writing_agent.execute(topic, outline, research_data)
        workflow_results["stages"]["writing"] = {
            "agent": "WritingAgent",
            "output": content,
            "status": "completed"
        }
        print(f"✓ Content drafted ({len(content)} characters)\n")

        # STAGE 4: Editing
        print("🔍 Stage 4: Editor Agent - Quality Review...")
        editor_results = self.editor_agent.execute(content)
        workflow_results["stages"]["editing"] = {
            "agent": "EditorAgent",
            "output": editor_results,
            "status": "completed"
        }
        print(f"✓ Editorial review completed\n")

        # Final Results
        workflow_results["end_time"] = datetime.now().isoformat()
        workflow_results["final_content"] = content
        workflow_results["quality_check"] = editor_results

        print(f"{'='*70}")
        print(f"✅ WORKFLOW COMPLETED SUCCESSFULLY")
        print(f"{'='*70}\n")

        return workflow_results

    def print_results(self, results: Dict[str, Any]):
        """Pretty print the workflow results"""
        print("\n" + "="*70)
        print("FINAL RESULTS")
        print("="*70)

        print(f"\nTopic: {results['topic']}")
        print(f"Start Time: {results['start_time']}")
        print(f"End Time: {results['end_time']}")

        print("\n--- RESEARCH FINDINGS ---")
        print(results["stages"]["research"]["output"])

        print("\n--- CONTENT OUTLINE ---")
        print(results["stages"]["outline"]["output"])

        print("\n--- FINAL CONTENT ---")
        print(results["final_content"])

        print("\n--- QUALITY CHECK ---")
        print(f"Grammar Check: {results['quality_check']['grammar_check']}")
        print(f"Editorial Review:\n{results['quality_check']['editorial_review']}")

        print("\n" + "="*70)



def example_2_full_pipeline():
    """Example 2: Full Content Creation Pipeline"""
    print("\n" + "="*70)
    print("EXAMPLE 2: Full Multi-Agent Content Pipeline")
    print("="*70)

    coordinator = CoordinatorAgent(llm)
    results = coordinator.orchestrate_content_creation(
        "The Impact of AI on Modern Education"
    )

    coordinator.print_results(results)


def example_3_custom_workflow():
    """Example 3: Custom workflow with specific agents"""
    print("\n" + "="*70)
    print("EXAMPLE 3: Custom Workflow - Research + Writing Only")
    print("="*70)

    # Use only specific agents
    research_agent = ResearchAgent(llm)
    writing_agent = WritingAgent(llm)

    topic = "Sustainable Energy Solutions"

    print(f"\n📚 Researching: {topic}")
    research = research_agent.execute(topic)

    print(f"\n✍️  Writing content...")
    # Create a simple outline for the writer
    simple_outline = f"""
    1. Introduction to {topic}
    2. Current challenges
    3. Innovative solutions
    4. Conclusion
    """

    content = writing_agent.execute(topic, simple_outline, research)

    print("\n--- FINAL CONTENT ---")
    print(content)
    print()





## Execution

In [ ]:
# Run examples
try:

    # Example 2: Full pipeline (production-like)
    example_2_full_pipeline()

    # Example 3: Custom workflow
    #example_3_custom_workflow()

except Exception as e:
    print(f"\n❌ Error: {e}")
    print("\nMake sure you have:")
    print("1. Set OPENAI_API_KEY environment variable")
    print("2. Installed required packages: pip install langchain langchain-openai")





EXAMPLE 2: Full Multi-Agent Content Pipeline

🤖 MULTI-AGENT CONTENT CREATION PIPELINE
Topic: The Impact of AI on Modern Education

📚 Stage 1: Research Agent - Gathering Information...
✓ Research completed (1635 characters)

📝 Stage 2: Outline Agent - Creating Structure...
✓ Outline created

✍️  Stage 3: Writing Agent - Drafting Content...
✓ Content drafted (3456 characters)

🔍 Stage 4: Editor Agent - Quality Review...
✓ Editorial review completed

✅ WORKFLOW COMPLETED SUCCESSFULLY


FINAL RESULTS

Topic: The Impact of AI on Modern Education
Start Time: 2025-12-09T05:27:33.630069
End Time: 2025-12-09T05:27:50.316968

--- RESEARCH FINDINGS ---
Research Summary:

1. Definition of AI: AI, or Artificial Intelligence, encompasses systems capable of performing tasks that typically require human intelligence. This includes a range of technologies such as machine learning, neural networks, and large language models.

2. Key Developments: In the realm of AI, significant advancements have been m

## Trouble shooting



### more details here
"""
📚 KEY CONCEPTS DEMONSTRATED:

1. MULTI-AGENT ARCHITECTURE
   - Specialized agents with clear responsibilities
   - Coordinator/Supervisor pattern
   - Tool calling for agent orchestration

2. CONTEXT ENGINEERING
   - Each agent gets relevant context
   - State passing between agents
   - Tool integration with agents

3. WORKFLOW ORCHESTRATION
   - Sequential agent execution
   - State management
   - Error handling and validation

⚠️ CHALLENGES IN MULTI-AGENT SYSTEMS (from slides):

1. COST
   - Multiple LLM calls increase expenses
   - Each agent invocation = API cost
   - Solution: Cache results, batch operations

2. LATENCY
   - Sequential agents = slower responses
   - Network overhead for each call
   - Solution: Parallel execution where possible

3. ERROR PROPAGATION
   - One agent's mistake affects downstream agents
   - Solution: Validation at each stage, error handling

4. DEBUGGING
   - Complex to trace issues across agents
   - Solution: Comprehensive logging, state tracking

5. COORDINATION
   - Agents need clear communication protocols
   - Solution: Well-defined interfaces, state schemas

🎯 WHEN TO USE MULTI-AGENT SYSTEMS:

✅ Good Use Cases:
   - Complex workflows (research → analysis → report)
   - Need for specialization (legal + medical + financial)
   - Quality control (agent reviews another's work)
   - Debate & consensus (multiple perspectives)

❌ Avoid When:
   - Simple, single-step tasks
   - Low latency requirements
   - Budget constraints

🔮 FUTURE DIRECTIONS:
   - Autonomous agents with minimal intervention
   - Collaborative problem-solving
   - Specialized expert agents
   - Self-improving agent teams
"""

# Examples

In [ ]:
# calculator agent
from langchain.agents import create_agent

# HumanMessage
from langchain_core.messages import HumanMessage

# tool
from langchain.tools import tool

@tool
def add(num1 , num2):
  """add two numbers"""
  print ("Add is called")
  return num1 + num2

@tool
def method1(num1 , num2):
  """subtract two numbers"""
  print ("method1 is called")
  return num1 + num2

@tool
def square(num1):
  """square a number"""
  print ("square is called")
  return num1 * num1

def create_calc_agent (llm):
  agent = create_agent(
    tools = [add, square, method1],
    model = llm
  )
  return agent

agent = create_calc_agent (llm)

result = agent.invoke({"messages": [HumanMessage( "what is 3-2")]})
print (result['messages'][-1].content)
#result = agent.invoke({"messages": [HumanMessage( "what is square of 9")]})
#print(result['messages'][-1].content)

method1 is called
3 - 2 is equal to 1.


In [ ]:
llm

ChatOpenAI(profile={'max_input_tokens': 16385, 'max_output_tokens': 4096, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': False, 'structured_output': False, 'image_url_inputs': False, 'pdf_inputs': False, 'pdf_tool_message': False, 'image_tool_message': False, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x797c8bac38f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x797c8bb15580>, root_client=<openai.OpenAI object at 0x797c8cb49670>, root_async_client=<openai.AsyncOpenAI object at 0x797c8c0f0080>, temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True)

calculator agent

@tool
def add(num1 , num2)

@tool
def square(num1)

